In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


In [11]:
df = pd.read_csv("Downloads/Attrition Dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv")

print("Shape:", df.shape)
df.head()


Shape: (1470, 35)


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [13]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [15]:
#Remove useless columns (identifiers & constants)
df.drop(['EmployeeNumber','EmployeeCount','StandardHours','Over18'], axis=1, inplace=True)

In [17]:
df['Attrition'] = df['Attrition'].map({'Yes':1, 'No':0})

In [21]:
cat_cols = df.select_dtypes(include='object').columns

encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le



In [23]:
X = df.drop("Attrition", axis=1)
y = df["Attrition"]


In [37]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [65]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

processed_test = pd.DataFrame(X_test, columns=X.columns)
processed_test["Attrition"] = y_test.values

processed_test.to_csv("test_processed.csv", index=False)

print("test_processed.csv saved!")


Train size: (1029, 30)
Test size: (441, 30)
test_processed.csv saved!


In [45]:
#!pip install xgboost

   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/101.7 MB 929.6 kB/s eta 0:01:49
   ---------------------------------------- 0.8/101.7 MB 1.1 MB/s eta 0:01:34
   ---------------------------------------- 1.0/101.7 MB 1.0 MB/s eta 0:01:39
    --------------------------------------- 1.3/101.7 MB 1.0 MB/s eta 0:01:38
    --------------------------------------- 1.8/101.7 MB 1.2 MB/s eta 0:01:21
    --------------------------------------- 2.4/101.7 MB 1.4 MB/s eta 0:01:10
   - -------------------------------------- 2.6/101.7 MB 1.5 MB/s eta 0:01:07
   - -------------------------------------- 3.4/101.7 MB 1.7 MB/s eta 0:00:59
   - -------------------------------------- 3.9/101.7 MB 1.8 MB/s eta 0:00:56
   - -------

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    matthews_corrcoef
)

import pandas as pd

models = {
    "Logistic Regression": LogisticRegression(max_iter=2000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}


In [51]:
results = []

for name, model in models.items():

    # Train
    model.fit(X_train, y_train)

    # Predict class
    y_pred = model.predict(X_test)

    # Predict probability (needed for AUC)
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X_test)[:,1]
    else:
        y_prob = model.decision_function(X_test)

    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    mcc = matthews_corrcoef(y_test, y_pred)

    results.append([name, accuracy, auc, precision, recall, f1, mcc])


C:\Users\ksree\anaconda3\Lib\site-packages\xgboost\training.py:200: UserWarning: [11:20:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:782: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [53]:
results_df = pd.DataFrame(results, columns=[
    "Model",
    "Accuracy",
    "AUC Score",
    "Precision",
    "Recall",
    "F1 Score",
    "MCC Score"
])

results_df.sort_values(by="F1 Score", ascending=False)


,Model,Accuracy,AUC Score,Precision,Recall,F1 Score,MCC Score
3,Naive Bayes,0.764172,0.759802,0.368000,0.647887,0.469388,0.354238
0,Logistic Regression,0.866213,0.807804,0.700000,0.295775,0.415842,0.396222
5,XGBoost,0.854875,0.735287,0.629630,0.239437,0.346939,0.325628
1,Decision Tree,0.768707,0.554853,0.261538,0.239437,0.250000,0.113740
2,KNN,0.843537,0.662752,0.571429,0.112676,0.188235,0.202208
4,Random Forest,0.825397,0.746022,0.363636,0.112676,0.172043,0.126338


In [63]:
import os
os.getcwd()

'C:\\Users\\ksree'

In [59]:
print(models)

{'Logistic Regression': LogisticRegression(max_iter=2000), 'Decision Tree': DecisionTreeClassifier(random_state=42), 'KNN': KNeighborsClassifier(), 'Naive Bayes': GaussianNB(), 'Random Forest': RandomForestClassifier(n_estimators=200, random_state=42), 'XGBoost': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,

In [61]:
import pickle
import os

os.makedirs("models", exist_ok=True)

for name, model in models.items():
    try:
        filename = name.lower().replace(" ", "_") + "_model.pkl"
        pickle.dump(model, open(os.path.join("models", filename), "wb"))
        print(f"Saved: {filename}")
    except Exception as e:
        print(f"Failed saving {name}:", e)

pickle.dump(scaler, open("scaler.pkl", "wb"))
print("Saved: scaler.pkl")


Saved: logistic_regression_model.pkl
Saved: decision_tree_model.pkl
Saved: knn_model.pkl
Saved: naive_bayes_model.pkl
Saved: random_forest_model.pkl
Saved: xgboost_model.pkl
Saved: scaler.pkl
